In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/v0.2.0.zip
!unzip v0.2.0.zip
%cd fastText-0.2.0
!make
!pip install .

In [4]:
import pandas as pd
import numpy as np

data = pd.read_csv('https://github.com/mbburova/MDS/raw/main/sentiment.csv', index_col=0)
data.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [5]:
import re
tag_regexp = re.compile("<[^>]*>")
regex = re.compile("[A-Za-z-]+")

def words_only(text, regex=regex):
    text = re.sub(tag_regexp, '', text)
    text = re.sub('\s+', ' ',text)
    text = re.sub(r'\\','', text)
    text = text.lower().strip()
    try:
        return " ".join(regex.findall(text))
    except:
        return ""


data['cleaned_review'] = data['review'].apply(words_only)
data['tokenized'] = data['cleaned_review'].apply(lambda x: x.split())
data.head()

,sentiment,review,cleaned_review,tokenized
0,1,With all this stuff going down at the moment w...,with all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,..."
1,1,"\The Classic War of the Worlds\"" by Timothy Hi...",the classic war of the worlds by timothy hines...,"[the, classic, war, of, the, worlds, by, timot..."
2,0,The film starts with a manager (Nicholas Bell)...,the film starts with a manager nicholas bell g...,"[the, film, starts, with, a, manager, nicholas..."
3,0,It must be assumed that those who praised this...,it must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai..."
4,1,Superbly trashy and wondrously unpretentious 8...,superbly trashy and wondrously unpretentious s...,"[superbly, trashy, and, wondrously, unpretenti..."


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *

In [7]:
X = [sentence.strip() for sentence in data.cleaned_review]
y = [str(label) for label in data.sentiment]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 5)

with open('ft_train.txt', 'w') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    
with open('ft_test.txt', 'w') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

print("Files are written")

Files are written


In [8]:
!./fasttext supervised -input ft_train.txt -output model -epoch 20

Read 1M words
Number of words:  54687
Number of labels: 2
Progress: 100.0% words/sec/thread: 1983561 lr:  0.000000 loss:  0.297761 ETA:   0h 0m


In [9]:
!./fasttext test model.bin ft_test.txt

N	2000
P@1	0.868
R@1	0.868


In [10]:
! pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
review = "This was such a great film! I am so lucky to watch it."

with open('review.txt', 'w') as outfile:
    outfile.write(review + '\n')

In [12]:
! cat review.txt

This was such a great film! I am so lucky to watch it.


In [13]:
!./fasttext predict model.bin review.txt

__label__1


In [14]:
!./fasttext predict-prob model.bin review.txt

__label__1 0.905953
